In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import glob
import pickle
from datetime import datetime, timedelta
import sys
from scipy.optimize import curve_fit
from matplotlib.lines import Line2D

#import the folder that has the analysis level class
sys.path.append("../CombinedData/")
sys.path.append("../AnalysisTools/")
import Dataset
import AnalysisTools

In [2]:
topdirs = {5:"../../data/Run5/", 6:"../../data/Run6/", 7:"../../data/Run7/", 8:"../../data/Run8/", 9:"../../data/Run9/"}
configs = {5:"../configs/run5_config.yaml", 6:"../configs/run6_config.yaml", 7:"../configs/run7_config.yaml", 8:"../configs/run8_config.yaml", 9:"../configs/run9_config.yaml"}
titles = {5:"SS uncoated", 6:"Refill of SS uncoated", 7:"MgF2 (50nm)", 8:"Pt (50nm)", 9:"MgF2 (20nm)"}
red_file_name = "combined_reduced.p"
#analysis tools objects, unloaded data
anas = {}
for n in topdirs:
    anas[n] = AnalysisTools.AnalysisTools(topdirs[n]+red_file_name, configs[n], title=titles[n], ramp_topdir=topdirs[n])



/Users/ejangelico/Documents/Stanford/GasBreakdownChamber/code/ad2-data-processing/notebooks/../AnalysisTools/AnalysisTools.py:127: UserWarning: genfromtxt: Empty input file: "../../data/Run8/ds03/g_events.txt"
  d = np.genfromtxt(os.path.join(root, self.config["g_events_name"]), delimiter=',', dtype=float)


# What does this notebook do? 
There are some instances where there are e-logged problems that occur in the run that get reconstructed as charge events but truly are not. This notebook takes combined reduced data and removes those events from the dataframe. So, be careful to check on this notebook when reprocessing reduced data. As the cuts used in this notebook may not be consistent if you change the charge reconstruction, for example. 

In [3]:
#This is a set of events in Run 7 that are from doing a 20 V/s ramp while at 
#bad high voltage ramp settings. I believe it was changing the HV every 0.5 second
#or so, whereas usually we do every 50 ms. As such, it was introducing enough high frequency
#coupling to couple into the charge channel. These were triggered on with an 8000 us window to 
#see what the noise was, and indeed it was a large ~ 2 mV pulse that gets reconstructed. 

anas[7].load_dataframe()
ana = anas[7]
df = ana.df
mask = (df["ch3 seconds"] > 1690577547.0) & (df["ch3 seconds"] < 1690577654.0)
#for each event that passes the mask, alter the ana.df to have ch3 charge set to None

for i in df[mask].index:
    ana.df.at[i, "ch3 charge"] = None

#resave the combined reduced file for this ana df
pickle.dump([ana.df], open(topdirs[7]+red_file_name, "wb"))